In [1]:
# Reference
# https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/control_your_robot.html#joint-control
# Figuring out the interface to the joints Go2
import genesis as gs

#Initializes Genesis with the CPU backend.
gs.init(backend=gs.cpu)

#Create a Scene
scene = gs.Scene(show_viewer=True)

#Adds a flat ground plane to the scene.
plane = scene.add_entity(gs.morphs.Plane())

#Integrate the Go2 Robot xml.
#robot = gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml')
robot = gs.morphs.MJCF(file="xml/Unitree_Go2/go2.xml")


#Add an entity to the scene.
Go2 = scene.add_entity(robot)

#Builds the scene.
scene.build()

[Genesis] [01:16:20] [INFO] ╭─────────────────────────────────────────────────────────────────────────────────────╮
[Genesis] [01:16:20] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉│
[Genesis] [01:16:20] [INFO] ╰─────────────────────────────────────────────────────────────────────────────────────╯
[Genesis] [01:16:22] [INFO] Running on [Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz] with backend gs.cpu. Device memory: 15.78 GB.
[Genesis] [01:16:22] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [01:16:22] [INFO] Scene <050aff8> created.
[Genesis] [01:16:22] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <c18516f>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [01:16:23] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <5439a4e>, morph: <gs.morphs.MJCF(file='C:\Users\ritwi\Desktop\total_robotics\genesis_AI_sims\Unitree_Go2\xml\Unitree_Go2\go2.xml')>, m

In [10]:
# Getting the joint Info Go2
joint_names = [
    'FL_hip_joint',
    'FL_thigh_joint',
    'FL_calf_joint',
    'FR_hip_joint',
    'FR_thigh_joint',
    'FR_calf_joint',
    'RL_hip_joint',
    'RL_thigh_joint',
    'RL_calf_joint',
    'RR_hip_joint',
    'RR_thigh_joint',
    'RR_calf_joint', 
]
motor_dofs = [i for name in joint_names for i in Go2.get_joint(name).dofs_idx_local]
motor_dofs

[0, 1, 2, 3, 4, 5, 10, 14, 7, 11, 15, 8, 12, 16, 9, 13, 17]

In [12]:
# Hard reset
import numpy as np
for i in range(150):
    if i < 50:
        Go2.set_dofs_position(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),motor_dofs)
    elif i < 100:
        Go2.set_dofs_position(motor_dofs,motor_dofs)
    else:
        Go2.set_dofs_position(motor_dofs,motor_dofs)

    scene.step()

[Genesis] [01:42:46] [INFO] Running at 4.08 FPS.
[Genesis] [01:42:46] [INFO] Running at 4.10 FPS.
[Genesis] [01:42:46] [INFO] Running at 4.16 FPS.
[Genesis] [01:42:46] [INFO] Running at 4.20 FPS.
[Genesis] [01:42:46] [INFO] Running at 4.23 FPS.
[Genesis] [01:42:47] [INFO] Running at 4.26 FPS.
[Genesis] [01:42:47] [INFO] Running at 4.31 FPS.
[Genesis] [01:42:47] [INFO] Running at 4.36 FPS.
[Genesis] [01:42:47] [INFO] Running at 4.41 FPS.
[Genesis] [01:42:47] [INFO] Running at 4.44 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.52 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.57 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.66 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.72 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.78 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.84 FPS.
[Genesis] [01:42:48] [INFO] Running at 4.89 FPS.
[Genesis] [01:42:49] [INFO] Running at 4.95 FPS.
[Genesis] [01:42:49] [INFO] Running at 4.99 FPS.
[Genesis] [01:42:49] [INFO] Running at 5.02 FPS.
[Genesis] [01:42:49]